Example Flipping a Coin

In [11]:
from typing import Tuple
import math

# Fliping a coin

In [12]:
def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Return mu and sigma corresponding to a Binomial(n, p)"""
    mu = p*n
    sigma = math.sqrt(p * (1-p) * n)
    return mu, sigma

In [13]:
from from_scratch import normal_cdf

def normal_probability_above(lo: float,
                             mu: float = 0,
                            sigma: float = 1) -> float:

                            """The probability that an N(mu, sigma) is greater then lo."""
                            return 1 - normal_cdf(lo, mu, sigma)

def normal_probability_between(lo: float,
                                hi: float,
                                mu: float = 0,
                                sigma: float = 1
                                ) -> float:

                            """the probability that an N(mu, sigma) is between lo and hi."""
                            return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)
                            
                            
#it's outside if it's not between 
def normal_probability_outside(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    """the probability that an  N(mu, sigma) is not between lo and hi"""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

P(both | older): 0.5007089325501317
P(both | either): 0.3311897106109325
0.2


In [14]:
from from_scratch import inverse_normal_cdf

def normal_upper_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    """return the z for which p(Z<= z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)


In [15]:

def normal_lower_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    """return the z for which P(Z>=z) = probability"""
    return inverse_normal_cdf(1 - probability, mu, sigma)


In [16]:
from from_scratch import inverse_normal_cdf
normal_lower_bound(3.2, 0, 2)

0.0

In [17]:
def normal_two_sided_bounds(probability: float, mu: float = 0, sigma: float = 1) -> Tuple[float, float]:
    """return the symmetric (about the mean) bounds that contain the specified probabillity"""
    tail_probability = (1 - probability) / 2
    #upper_bound should have tail probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return  lower_bound, upper_bound

In [18]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [19]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

In [20]:
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

1.0


In [21]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

# p-value

In [46]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """how likely are we to see a value at least as extreme as x (in either direction) if our values are from an N(mu, sigma)?"""
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
        else:
            return 'below'

SyntaxError: invalid syntax (<ipython-input-46-4bee38f901b4>, line 5)

In [23]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

In [24]:
import random

In [25]:
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0
                    for _ in range(1000))

    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
    
assert 59 < extreme_value_count < 65, f"{extreme_value_count}"


In [26]:
two_sided_p_value(531, mu_0, sigma_0)

0.04992428403969762

In [27]:
upper_p_value = normal_probability_above

In [28]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582072

In [29]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

# Confidence Intervals

In [30]:
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

In [31]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.525, 0.525)

In [32]:
p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
normal_two_sided_bounds(0.95, mu, sigma)

(0.54, 0.54)

# p-Hacking

In [33]:
from typing import List

def run_experiment() -> List[bool]:
    """flips a fair coin 1000 times, true = head, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 468 or num_heads > 531


In [34]:
random.seed(0)

In [35]:
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiments for experiment in experiments
if reject_fairness(experiment)])

In [36]:
assert num_rejections == 44

# Running an A/B test

In [37]:
from typing import Tuple
import math

def estimated_parameters(N: int, n: int) -> Tuple[float,float]:
    p = n/N
    sigma = math.sqrt(p* (1-p) / N)
    return p, sigma

In [38]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

In [52]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [51]:
two_sided_p_value(z)

In [55]:
z = a_b_test_statistic(1000, 200, 1000, 300)
two_sided_p_value(z)

2.006305070967329e-07